In [1]:
import os
from PIL import Image
import numpy as np

In [2]:
import cv2
from skimage import img_as_ubyte, io
def block_no_expand(name):
    im=np.array(Image.open(name))
    im_lab=cv2.cvtColor(im,cv2.COLOR_RGB2HSV)
    im_lab[:,:,-1]=np.interp(im_lab[:,:,-1],(0,255),(0,255/4))
    im_rgb=cv2.cvtColor(im_lab,cv2.COLOR_HSV2RGB)
    im_ht=Image.fromarray(im_rgb).convert("1")
    im_ht=img_as_ubyte(im_ht)
    # gray_level(im_ht,(2,4),True)
    return im_ht
 

In [3]:
def basic_rg_vcs(img_name="lena.png", to_save=None, n=8):
    img_arr = np.invert(img_name.convert("1"))
    shares = np.zeros((n,)+img_arr.shape, bool)
    for (i, j), p in np.ndenumerate(img_arr):
        if p:
            for ind in range(n):
                shares[ind, i, j] = np.random.choice([True, False])
        else:
            v = np.random.choice([True, False])
            shares[:, i, j] = v
    dec = shares[0] & shares[1] & shares[2] & shares[3] & shares[4] & shares[5] & shares[6] & shares[7]
    if to_save:
        Image.fromarray(dec).save(to_save)
    else:
        return dec

In [4]:
from skimage.metrics import peak_signal_noise_ratio

def re_halftone(pre, path, path_ori):
    # Load image
    #orig=path
    
    img=img_as_ubyte(Image.open(path))
    pil_halftone = Image.open(path).convert('L').convert("1")
    halftone = img_as_ubyte(pil_halftone)
    ref = img_as_ubyte(Image.open(path_ori).convert('L').convert("1"))
    def IHT(radius=3,sigma_s=100,cnts=3):
        dehalftoned=halftone.copy()
        for i in range(cnts):
            dehalftoned = cv2.bilateralFilter(I,radius,sigma_s,sigma_s,dst=dehalftoned)
        return dehalftoned

    best_psnr = 0
    best_image = None
    best_ehist = None
    for radius in range(1, 15,2): # 超参数
        for sigma_s in [10,20,30,50,100,150,200,400,500,600]: # 超参数
            for cnts in range(1,9): # 超参数
                I=cv2.GaussianBlur(halftone.copy(),(radius,radius),0)
                dehalftoned=IHT(radius,sigma_s,cnts)
                ehist=cv2.equalizeHist(dehalftoned) # ehist 为增强解密图像
                name=f"{pre}/{cnts}_{radius}_{sigma_s}.png"
                psnr = peak_signal_noise_ratio(ehist,ref)
                if psnr > best_psnr:
                    best_psnr = psnr
                    best_image = name
                    best_ehist = ehist

    print("Best PSNR:", best_psnr)
    print("Best image:", best_image)
    return best_ehist

In [5]:
ori = './ori'
for i in os.listdir(ori):
    if i.endswith('.jpg'):
        os.rename(os.path.join(ori,i),os.path.join(ori,i[0:3]+'.jpg'))

In [9]:
for i in os.listdir(ori):
    path = os.path.join(ori,i)
    Image.open(path).resize((224,224)).save(path)

In [10]:
nx = './nx'
for i in os.listdir(ori):
    path = os.path.join(ori,i)
    save_path = os.path.join(nx,i)
    Image.open(save_path).resize((224,224)).save(save_path)
    #Image.fromarray(block_no_expand(path)).convert('RGB').save(save_path)

In [11]:
re = './re'
for i in os.listdir(nx):
    path = os.path.join(nx,i)
    save_path = os.path.join(re,i)
    t = re_halftone(re, path, os.path.join(ori,i))
    Image.open(save_path).resize((224,224)).save(save_path)
    #Image.fromarray(t).convert('RGB').save(save_path)

Best PSNR: 8.22318306416508
Best image: ./re/1_9_10.png
Best PSNR: 8.182241092894504
Best image: ./re/1_5_100.png
Best PSNR: 6.900801105485661
Best image: ./re/1_5_200.png
Best PSNR: 7.384308278182804
Best image: ./re/1_5_200.png
Best PSNR: 6.081772744865076
Best image: ./re/1_7_10.png
Best PSNR: 5.945372249220701
Best image: ./re/1_9_10.png
Best PSNR: 7.429615047363879
Best image: ./re/1_5_400.png
Best PSNR: 8.617458017718695
Best image: ./re/1_3_50.png
Best PSNR: 7.807228137795963
Best image: ./re/1_7_600.png
Best PSNR: 6.662052848383054
Best image: ./re/1_5_500.png
Best PSNR: 6.187229759626821
Best image: ./re/1_13_10.png
Best PSNR: 8.278089825914206
Best image: ./re/1_9_10.png
Best PSNR: 6.458577238220613
Best image: ./re/1_9_10.png
Best PSNR: 6.18407041614066
Best image: ./re/1_7_10.png
Best PSNR: 6.802897528341161
Best image: ./re/1_7_150.png
Best PSNR: 6.992419123008729
Best image: ./re/1_9_10.png
